In [8]:
import pandas as pd
import json
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import openai
import random
from retrying import retry
import pyarrow as pa
import pyarrow.parquet as pq
from transformers import AutoTokenizer

openai.api_base = "https://api.deepseek.com"


class OpenAIGPT:
    def __init__(self, model_name="deepseek-chat", keys_path=None):
        self.model_name = model_name
        with open(keys_path, encoding="utf-8", mode="r") as fr:
            self.keys = [line.strip() for line in fr if len(line.strip()) >= 4]

    def __post_process(self, response):
        return response["choices"][0]["message"]["content"]

    @retry(wait_fixed=300, stop_max_attempt_number=50)
    def __call__(self, message):
        if message is None or message == "":
            return False, "Your input is empty."

        # current_key = random.choice(self.keys)
        current_key = self.keys[0] if len(self.keys) == 1 else random.choice(self.keys)
        openai.api_key = current_key
        response = openai.ChatCompletion.create(
            model=self.model_name,
            messages=[{"role": "user", "content": message}],
            temperature=0.3,
            top_p=0.7,
            frequency_penalty=0.6,
            presence_penalty=0.6,
            n=1,
        )
        return self.__post_process(response)


igpt = OpenAIGPT(keys_path="apikeys.txt")

In [9]:
print(igpt('现在是2023-04-14 09:00:00+08:00\n前五个交易日上证指数的价格如下3302.751,3312.481,3331.505,3317.077,3316.862\n\n2023-03-29 07:34:27 新闻:【财险龙头新能源车险保费大增 降赔付率依然任重道远】 据证券日报，2022年，太保产险、人保财险新能源车险保费同比分别大幅增长超90%、72%，这意味着财险巨头新能源车险保费正在加速增长，也在一定程度上反映出当前新能源车险市场的发展势头。业内人士认为，新能源汽车的渗透率仍在持续提高，新能源车险必然是未来车险市场增量的主力，险企也在加大布局力度。不过，如何控制新能源车险的赔付率、中小险企如何涉足新能源车险等问题依然需要持续探索，而造车新势力进军新能源车险的影响也有待持续观察。\n\n时间：2023-04-09 17:01:10\n标题：三一重工股份有限公司2022年社会责任报告（英文版）\n\n\n 2022年三一重工股份有限公司社会责任报告摘要如下：\n\n- 公司坚持“质量改变世界”的使命，致力于制造世界一流的产品，连续三年获得国家科学技术进步奖，两次获得国家技术发明奖，代表了中国工程机械行业最高荣誉。\n\n- 实施“数字化、电气化和国际化”战略，推进智能制造和绿色制造，2022年新增18个智能工厂，平均产能提升68%，25个工厂实现双碳目标。\n\n- 加强供应商管理，100%供应商签署合作承诺书，实现绿色供应链。\n\n- 加大研发投入，研发投入100.26亿元，占营业收入的9.78%，申请专利2663件，获中国专利银奖3项。\n\n- 关注员工成长，建立职业发展通道，提供培训和激励，员工满意度87%。\n\n- 支持乡村振兴，投资教育、医疗、工业项目，帮助改善基础设施和生态环境。\n\n- 积极参与公益事业，开展志愿服务、环保、扶贫等活动，员工志愿服务时间161.17万小时。\n\n- 加强企业治理，完善风险管控和内控制度，强化董事会建设，开展廉洁教育，公司治理水平不断提升。\n\n时间：2023-04-10 18:40:11\n标题：三一重工股份有限公司关于以集中竞价交易方式首次回购股份公告\n\n\n 三一重工股份有限公司（以下简称“公司”）通过集中竞价交易方式回购股份，用于后续实施员工持股计划。本次回购价格不低于人民币 5 亿元，不超过人民币 8 亿元，回购期限为自公司董事会审议通过回购方案之日起不超过 45 天。公司于 2023 年 4 月 10 日通过集中竞价交易方式首次回购股份 6,964,985 股，已回购股份占公司总股本的比例为 0.08%，购买的最高价为 17.10 元/股，最低价为 16.99 元/股，交易总金额为 11,869.07 万元（不含交易费用）。本次回购符合法律法规、规范性文件的有关规定及公司回购股份方案。公司后续将根据回购股份进展情况及时履行信息披露义务，敬请广大投资者理性投资并注意投资风险。\n\n时间：2023-04-13 23:23:47\n标题：三一重工股份有限公司2022年年度股东大会会议资料\n\n\n 这份文件是三一重工股份有限公司2022年年度股东大会会议资料，主要包括以下内容：\n\n1. 2022年度董事会工作报告，汇报了公司2022年的经营情况、发展战略、数智化转型、电动化转型、国际化转型、研发创新、人力资源建设等方面的进展。\n\n2. 2022年度监事会工作报告，总结了监事会2022年的工作情况和独立意见。\n\n3. 2022年年度报告及报告摘要。\n\n4. 2022年度财务决算报告。\n\n5. 2022年度利润分配预案。\n\n6. 关于2022年度董监薪酬考核的议案。\n\n7. 关于向银行申请授信额度的议案。\n\n8. 关于预计2023年度日常关联交易的议案。\n\n9. 未来三年股东回报规划(2023-2025年)。\n\n10. 关于续聘2023年度会计师事务所的议案。\n\n11. 关于开展金融衍生品业务的议案。\n\n12. 关于使用自有闲置资金购买理财产品的议案。\n\n13. 关于申请统一注册发行多品种债务融资工具(DFI)的议案。\n\n14. 2022年度独立董事述职报告。三一重工股票的5天（短期）后对数收益率属于负面，请进一步分析，给出你的投资建议，你应该作出以下选择： A. 中性 B. 负面 C. 极度负面'))

为了给出投资建议，我们需要分析提供的信息并考虑市场情况。以下是对所提供信息的分析：

1. **上证指数价格**：前五个交易日的上证指数价格显示了一个小幅波动，但整体趋势相对平稳。这表明市场整体可能处于一个调整或观望的状态。

2. **新能源车险保费增长新闻**：这条新闻显示了新能源车险市场的快速增长，这对相关保险公司和整个汽车行业可能是一个积极的信号。然而，新闻也提到了赔付率控制的问题，这可能会影响保险公司的盈利能力。

3. **三一重工社会责任报告**：报告显示三一重工在多个方面取得了显著成就，包括技术创新、智能制造、绿色制造、研发投入和员工满意度等。这些成就可能会增强投资者对公司未来发展的信心。

4. **回购股份公告**：公司通过集中竞价交易方式回购股份，表明公司对自身股价的信心以及对未来发展的积极预期。这种行为通常被视为积极的信号，因为它可以提高每股收益并减少流通股本。

5. **年度股东大会会议资料**：这份资料提供了公司的详细经营情况和未来的发展计划，包括财务状况、利润分配预案和未来的战略规划等。这些信息对于评估公司的长期价值非常重要。

综合以上信息，虽然短期内上证指数的表现相对平稳，但三一重工的积极表现（如社会责任报告中的成就和回购股份的行为）可能预示着公司长期的正面发展潜力。因此，基于目前的信息和对数收益率的负面表现可能是短期的市场波动或调整的结果，并不一定反映公司的基本面变化。

投资建议：A. 中性 - 考虑到短期内市场的波动性和三一重工的长期潜力，建议保持中性的投资态度，继续观察市场动态和公司的进一步发展情况再做决策。
